# Load and Clean Data

In [1]:
# import nltk
# nltk.download("stopwords")

ModuleNotFoundError: No module named 'nltk'

In [2]:
import pandas as pd
#from nltk.corpus import stopwords
df = pd.read_json("../standards/iso_flat.json")
df.head()

,current_status,datetime,description,edition,ics,id,number_of_pages,preview_url,publication_date,section_titles,sections,tc,title,url
0,Published,2018-03-10 19:40:16,The method is applicable to carbide and binder...,Edition : 1,[77.160],ISO 4501:1978,Number of pages : 3,https://www.iso.org/obp/ui/#!iso:std:10399:en,1978-08,"[FOREWORD, 1 SCOPE]",[FOREWORD\nISO (the International Organization...,ISO/TC 119/SC 4,Hardmetals -- Determination of titanium -- Pho...,https://www.iso.org/standard/10399.html
1,Published,2018-03-10 19:40:24,"This part presents definitions of terms, in En...",Edition : 2,"[83.160.01, 01.040.83]",ISO 4223-2:1991,Number of pages : 2,https://www.iso.org/obp/ui/#!iso:std:10022:en,1991-01,"[Foreword, Scope, 1 General definitions, 2 ...",[Foreword\nISO (the International Organization...,ISO/TC 31,Definitions of some terms used in the tyre ind...,https://www.iso.org/standard/10022.html
2,Published,2018-03-10 19:40:32,The determination is carried out in the follow...,Edition : 1,[71.100.40],ISO 4314:1977,Number of pages : 2,https://www.iso.org/obp/ui/#!iso:std:10183:en,1977-02,"[FOREWORD, 1 SCOPE AND FIELD OF APPLICATION,...",[FOREWORD\nISO (the International Organization...,ISO/TC 91,Surface active agents -- Determination of free...,https://www.iso.org/standard/10183.html
3,Published,2018-03-10 19:40:40,,Edition : 1,"[25.220.50, 67.250, 97.040.60]",ISO 4531-2:1998,Number of pages : 3,https://www.iso.org/obp/ui/#!iso:std:10443:en,1998-10,"[Foreword, Introduction, 1 Scope, 2 Normat...",[Foreword\nISO (the International Organization...,ISO/TC 107,Vitreous and porcelain enamels -- Release of l...,https://www.iso.org/standard/10443.html
4,Published,2018-03-10 19:40:48,The water in a test portion is removed by azeo...,Edition : 2,[71.100.40],ISO 4318:1989,Number of pages : 2,https://www.iso.org/obp/ui/#!iso:std:10189:en,1989-06,"[Foreword, 1 Scope, 2 Normative references]",[Foreword\nISO (the International Organization...,ISO/TC 91,Surface active agents and soaps -- Determinati...,https://www.iso.org/standard/10189.html


In [3]:
def get_field(row):
    """
    The field is the first set of numbers before the decimal in in the ics code of form ##.###,#
    :param row, a row of a dataframe containing the field "ics"
    :return field, a list of the fields the standard belongs to
    """
    field = []
    for ics in row:
        field.append(ics.split(".")[0])
    return field

def get_group(row):
    group = []
    for ics in row:
        group.append(ics.split(".")[1])
    return group

def get_subgroup(row):
    subgroup = []
    for ics in row:
        try:
            subgroup.append(ics.split(".")[2])
        except IndexError:
            pass

In [4]:
df["field"] = df["ics"].apply(lambda x: get_field(x))
df["group"] = df["ics"].apply(lambda x: get_group(x))
df["subgroup"] = df["ics"].apply(lambda x: get_subgroup(x))

In [5]:
#check the number of samples in each field
import itertools, collections
counter = collections.Counter(itertools.chain(*list(df["field"])))
print(counter)

Counter({'35': 4512, '25': 2333, '13': 2245, '01': 1561, '83': 1465, '11': 1282, '43': 1239, '23': 1203, '77': 1053, '91': 1003, '03': 958, '17': 938, '49': 906, '59': 896, '67': 875, '75': 804, '71': 753, '37': 728, '65': 701, '21': 619, '47': 531, '53': 514, '27': 416, '97': 392, '87': 354, '81': 340, '73': 307, '07': 300, '55': 253, '85': 235, '79': 223, '93': 184, '19': 168, '61': 124, '33': 121, '31': 64, '39': 58, '29': 40, '45': 30, '95': 8})


Won't predict well for fields 29, 95

In [6]:
#get the relevant text for predictors and clean text
import re

def get_scope(row):
    text = ""
    for item in row:
        item = item.lower()
        pattern = re.compile("scope")
        match = re.search(pattern, item)
        if match:
            text = item.replace("scope and field application", "")
            text = item.replace("scope", "")
    return text
    
df["scope"] = df["sections"].apply(lambda x: get_scope(x))

In [7]:
df["text"] = df["scope"] + " " + df["description"]

In [8]:
#check for any null fields
any(pd.isnull(df["text"]))


False

In [9]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [10]:
stop_words = [u"the", u"this", u"scope", u"international", u"iso", u"iec", u"requirement",
              u"standard", u"application", u"specifies"]
#stop = set(stopwords.words('english'))
for word in stop_words:
    nlp.vocab[word].is_stop = True

In [11]:
def clean_and_tokenize(text):
    text = text.lower()
    parsed = nlp(text)
    words = [token.lemma_ for token in parsed if not token.is_stop and not token.is_punct and not
            token.like_num and not token.is_space]
    return " ".join(words)


In [ ]:
df["parsed_text"] = df["text"].apply(lambda x: clean_and_tokenize(x))

In [ ]:
for item in df["parsed_text"][0:5]:
    print(item)

# Print the top words and their weights

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

def get_tfidf_series(text_list):
    vectorizer = TfidfVectorizer()
    values = vectorizer.fit_transform(text_list)
    feature_names = vectorizer.get_feature_names()
    for i in range(len(feature_names)):
        feature_names[i] = feature_names[i].decode('latin1')
    tfidf_series = pd.Series(values.toarray().sum(axis=0), index = feature_names).sort_values(ascending = False)
    return list(zip(feature_names, tfidf_series))

In [ ]:
weighted_words = get_tfidf_series(df["parsed_text"])
top_50 = sorted(weighted_words, key = lambda x: x[1])[0:50]
# top_50 = [list(t) for t in zip(*top_50)]
# plt.barh(top_50[0], top_50[1], figsize = (8,20))
# plt.show()
top_50

# Classification

In [ ]:
#transform the label to be predicted (field) to a binary representation
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df["field"].values)
labels = list(mlb.classes_)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import cross_val_score #automatically uses stratified split for binary/multiclass data
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_fscore_support

def score_summary(clf, features, classification):
    accuracy = cross_val_score(clf, features, classification, cv = 5, scoring = 'accuracy')
    f1 = cross_val_score(clf, features, classification, cv = 5, scoring = 'f1')
    precision = cross_val_score(clf, features, classification, cv = 5, scoring = 'precision')
    recall = cross_val_score(clf, features, classification, cv = 5, scoring = 'recall')
    print("accuracy: %0.2f (+/- %0.2f)" % (accuracy.mean(), accuracy.std()*2))
    print("F1 score: %0.2f (+/- %0.2f)" % (f1.mean(), f1.std()*2))
    print("precision: %0.2f (+/- %0.2f)" % (precision.mean(), precision.std()*2))
    print("recall: %0.2f (+/- %0.2f)" % (recall.mean(), recall.std()*2))
    return [accuracy.mean(), f1.mean(), precision.mean(), recall.mean()]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = Pipeline([
    ('vectorizer', TfidfVectorizer),
    ('clf', OneVsRestClassifier(RandomForestClassifier()))])

In [ ]:
score_summary(classifier, df["parsed_text"], df["field"])